<a href="https://colab.research.google.com/github/Bijan-K/QWEN2-7b-notebook/blob/main/QWEN2_farsi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Installing model dependences
!pip install -q transformers accelerate bitsandbytes torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 53.1 MB/s eta 0:00:00


In [ ]:
# @title Downloading the Model
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# @title Setting up the model
messages = [
    {"role": "system", "content": "شما یک دستیار کمک کننده هستید."},
]

def run_model(messages):
  text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
  )
  model_inputs = tokenizer([text], return_tensors="pt").to(device)

  generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512 #Change this to your needs
  )
  generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  messages.append({"role": "assistant", "content" : f"{response}" })
  return response


In [ ]:
# @title Enter your prompt
prompt = "سلام روز شما چطور است؟"  # @param {type:"string"}

messages.append({"role": "user", "content": prompt})

print("-"*10 + " Answer " + "-" *10)
print(run_model(messages))

---------- Answer ----------
سلام! روز من خوب است، متشکرم. چگونه می توانم به شما کمک کنم؟


In [ ]:
# @title Very Simple RAG

Doc = "اِلِکترون یک ذره زیر اتمی‌‌ست با بار الکتریکی منفی و برابر با بار بنیادی. الکترون‌ها به نسل نخست از خانواده لپتون‌ها تعلق دارند و به ‌طور عمومی به عنوان ذره بنیادی شناخته می‌شوند زیرا هیچ جزء و زیرساختار تشکیل‌دهنده شناخته‌شده‌ای ندارند. الکترون جرمی تقریباً برابر با یک بر روی ۱۸۳۶ جرم پروتون دارد. ویژگی‌های کوانتومی الکترون شامل تکانه زاویه‌ای ذاتی (اسپین) با مقدار نیمه‌صحیح بر حسب ثابت کاهیدهٔ پلانک است و این یعنی الکترون یک نوع فرمیون است. به دلیل فرمیون بودن، طبق اصل طرد پاولی، دو الکترون مختلف نمی‌توانند حالات کوانتومی یکسانی را اشغال کنند. " # @param {type:"string"}


messages.append({"role": "user", "content": f"از اطلاعات داده شده به سوالات بعدی پاسخ بده : {Doc}"})






In [ ]:
# @title Enter your prompt
prompt = "متنی که بهت داده شده را توضیح بده"  # @param {type:"string"}

messages.append({"role": "user", "content": prompt})

print("-"*10 + " Answer " + "-" *10)
print(run_model(messages))

---------- Answer ----------


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.02 GiB. GPU 

In [ ]:
# @title Reset converstaion

messages = [{"role": "system", "content": "شما یک دستیار کمک کننده هستید."},]
